In [ ]:
import pickle
import networkx as nx
import matplotlib.pyplot as plt

# Load the .pkl file
with open('./my_model_graph1.pkl', 'rb') as f:
    data = pickle.load(f)

# Extract data
edge_index = data['edge_index']
face_features = data['face_features']
# edge_features = data['edge_features'] # This key also doesn't exist

# Create a graph from the edge_index
G = nx.Graph()
G.add_edges_from(edge_index.T) # Transpose edge_index to get a list of tuples

# Draw the graph
pos = nx.spring_layout(G, seed=42)  # Spring layout for nice spacing
plt.figure(figsize=(8, 6))
nx.draw(G, pos, with_labels=False, node_color='skyblue', edge_color='gray', node_size=800, font_size=12)

# Optional: Add labels
labels = {i: f"F{i}" for i in G.nodes}
nx.draw_networkx_labels(G, pos, labels, font_size=10)

plt.title("B-Rep Face Adjacency Graph")
plt.axis('off')
plt.show()

In [ ]:
import pickle

with open("my_model_graph.pkl", "rb") as f:
    data = pickle.load(f)



In [ ]:
print("Keys in data:", data.keys())


In [ ]:
import pandas as pd

# Convert and display face features
if 'face_features' in data:
    face_df = pd.DataFrame(data['face_features'])
    print("Face Features:")
    display(face_df)

# Convert and display edge features
if 'brep_edge_features' in data:
    edge_df = pd.DataFrame(data['brep_edge_features'])
    print("Edge Features:")
    display(edge_df)


In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

# Load the graph data from the .pkl file
with open('my_model_graph1.pkl', 'rb') as f:
    data = pickle.load(f)

# Access A1 shortest path matrix
A1 = data['proximity_A1_shortest_path']

print("Proximity A1: Shortest Path Matrix (Face-to-Face):")
print(A1)

# Optional: Visualize it as a heatmap
plt.imshow(A1, cmap='viridis')
plt.colorbar(label='Shortest Path Length')
plt.title('Proximity A1: Shortest Path Between Faces')
plt.xlabel('Face Index')
plt.ylabel('Face Index')
plt.show()


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
print(data.keys())


In [ ]:
import torch
from torch_geometric.data import Data
import numpy as np

# Assuming face_features is a dictionary of numpy arrays as shown in the notebook state
face_features_dict = data['face_features']

# Convert the dictionary of numpy arrays to a single numpy array by stacking them column-wise
# Ensure all arrays have the same number of rows (nodes)
feature_arrays = [face_features_dict[key] for key in face_features_dict]
# Check if all arrays have the same number of rows
if all(arr.shape[0] == feature_arrays[0].shape[0] for arr in feature_arrays):
    face_features_np = np.stack(feature_arrays, axis=1)
else:
    raise ValueError("Feature arrays in 'face_features' have different numbers of rows.")

# Convert the combined numpy array to a torch tensor
x_tensor = torch.tensor(face_features_np, dtype=torch.float)

# Access edge index using the dictionary key and transpose
edge_index_tensor = torch.tensor(data['edge_index'], dtype=torch.long).T

graph_data = Data(
    x=x_tensor,  # node features
    edge_index=edge_index_tensor # edge index must be [2, num_edges]
)

print(graph_data)

In [ ]:
import torch.nn as nn

class FaceEmbeddingMLP(nn.Module):
    def __init__(self, in_dim=3, hidden_dim=32, out_dim=64):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, x):
        return self.mlp(x)


In [ ]:
import torch
from torch_geometric.data import Data
import numpy as np

# Recreate graph_data with original features
face_features_dict = data['face_features']
feature_arrays = [face_features_dict[key] for key in face_features_dict]
if all(arr.shape[0] == feature_arrays[0].shape[0] for arr in feature_arrays):
    face_features_np = np.stack(feature_arrays, axis=1)
else:
    raise ValueError("Feature arrays in 'face_features' have different numbers of rows.")
x_tensor = torch.tensor(face_features_np, dtype=torch.float)
# Access edge index using the dictionary key and DO NOT transpose
edge_index_tensor = torch.tensor(data['edge_index'], dtype=torch.long)

graph_data = Data(
    x=x_tensor,  # node features
    edge_index=edge_index_tensor # edge index must be [2, num_edges]
)

# Initialize the embedding layer with the correct input dimension
embedding_layer = FaceEmbeddingMLP(in_dim=graph_data.x.shape[1])
x_embedded = embedding_layer(graph_data.x)

# Optionally update graph_data.x with the embedded features
graph_data.x = x_embedded

print("Original graph_data.x shape:", x_tensor.shape)
print("Embedded graph_data.x shape:", x_embedded.shape)
print("graph_data.edge_index shape:", graph_data.edge_index.shape)

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.data import Data

# Define a simple GAT model
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=1):
        super(GAT, self).__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=heads)
        self.gat2 = GATConv(hidden_channels * heads, out_channels, heads=1)  # final heads = 1

    def forward(self, x, edge_index):
        x = self.gat1(x, edge_index)
        x = F.elu(x)
        x = self.gat2(x, edge_index)
        return x

# Create the model
model = GAT(in_channels=64, hidden_channels=32, out_channels=64, heads=4)

# Forward pass
out = model(graph_data.x, graph_data.edge_index)
print("Output node embeddings shape:", out.shape)


In [ ]:
from sklearn.cluster import KMeans
import numpy as np

# Fix for the RuntimeError
embeddings_np = np.array(out.detach().cpu().tolist())

# Run KMeans clustering
kmeans = KMeans(n_clusters=3, random_state=0).fit(embeddings_np)
face_labels = kmeans.labels_

print("Cluster labels for each face:", face_labels)


In [ ]:
for i, label in enumerate(face_labels):
    print(f"Face {i}: Cluster {label}, Type: {face_features_dict['type'][i]}, Area: {face_features_dict['area'][i]}")

In [ ]:
import h5py

file = h5py.File("/content/training_MFCAD++.h5", "r")
print(list(file.keys()))  # Should show node features, edge index, labels, etc.


In [ ]:
import h5py

# Load the dataset file
file_path = "/content/training_MFCAD++.h5"
with h5py.File(file_path, "r") as f:
    print("Top-level keys (each CAD sample):")
    print(list(f.keys()))  # ['0', '1', '2', ..., '1999']

    # Let's inspect the structure under the first sample
    sample_key = '0'
    print(f"\nKeys under '{sample_key}':")
    print(list(f[sample_key].keys()))


In [ ]:
import h5py

with h5py.File('/content/training_MFCAD++.h5', 'r') as f:
    keys = list(f.keys())
    print("Number of entries:", len(keys))
    first_key = keys[0]
    print("First key:", first_key)
    print("Available subkeys in group:")
    print(list(f[first_key].keys()))


In [ ]:
class MFCADDataset(torch.utils.data.Dataset):
    def __init__(self, h5_file):
        self.h5_file = h5_file
        self.file = h5py.File(h5_file, 'r')
        self.keys = list(self.file.keys())

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        key = self.keys[idx]
        group = self.file[key]

        # Node features
        x = torch.tensor(group['V_1'][:], dtype=torch.float)

        # Edges
        edge_index = torch.tensor(group['E_1_idx'][:], dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(group['E_1_values'][:], dtype=torch.float)

        # Labels
        y_np = group['labels'][:]
        y = torch.tensor(y_np.astype(int), dtype=torch.long)

        return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)


In [ ]:
dataset = MFCADDataset("/content/training_MFCAD++.h5")
print(f"Number of graphs: {len(dataset)}")

sample = dataset[0]
print(sample)



In [ ]:
with h5py.File('/content/training_MFCAD++.h5', 'r') as f:
    group = f['0']
    print("V_1:", group['V_1'][:].shape)
    print("V_2:", group['V_2'][:].shape)
    print("E_1_idx:", group['E_1_idx'][:].shape)
    print("E_1_values:", group['E_1_values'][:].shape)
    print("facet_labels:", group['facet_labels'][:].shape)
    print("labels:", group['labels'][:].shape)


In [ ]:
from torch_geometric.utils import to_networkx
import matplotlib.pyplot as plt
import networkx as nx

data = dataset[0]
G = to_networkx(data, to_undirected=True)
plt.figure(figsize=(8, 6))
nx.draw(G, node_size=20)
plt.title("Sample Graph")
plt.show()


In [ ]:
from torch_geometric.utils import train_test_split_edges

def create_masks(data, train_ratio=0.7, val_ratio=0.15):
    num_nodes = data.num_nodes
    indices = torch.randperm(num_nodes)

    train_end = int(train_ratio * num_nodes)
    val_end = train_end + int(val_ratio * num_nodes)

    data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    data.val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    data.train_mask[indices[:train_end]] = True
    data.val_mask[indices[train_end:val_end]] = True
    data.test_mask[indices[val_end:]] = True
    return data


In [ ]:
data = create_masks(dataset[0])


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=8, dropout=0.6):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout)
        self.conv2 = GATConv(hidden_channels * heads, hidden_channels, heads=heads, dropout=dropout)
        self.conv3 = GATConv(hidden_channels * heads, hidden_channels, heads=heads, dropout=dropout)
        self.conv4 = GATConv(hidden_channels * heads, out_channels, heads=1, concat = False, dropout=dropout)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.3, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.3, training=self.training)
        x = F.elu(self.conv2(x, edge_index))
        x = F.dropout(x, p=0.3, training=self.training)
        x = self.conv4(x, edge_index)
        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define in_channels and num_classes
# in_channels for GAT should be the output dimension of the embedding layer
in_channels = 64
num_classes = 25 # Number of unique classes in your labels

# Change model to GAT
model = GAT(in_channels=in_channels, hidden_channels=64, out_channels=num_classes, heads=8).to(device) # Using GAT model
data = data.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=5e-4)


# Assuming embedding_layer is defined and available from a previous cell (e.g., KqvMh4bpKm7n)

def train():
    model.train()
    optimizer.zero_grad()

    # Apply embedding layer to the node features
    embedded_x = embedding_layer(data.x)

    out = model(embedded_x, data.edge_index)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()


def test():
    model.eval()
    with torch.no_grad(): # Use no_grad for evaluation
        # Apply embedding layer to the node features
        embedded_x = embedding_layer(data.x)

        out = model(embedded_x, data.edge_index)
        pred = out.argmax(dim=1)

        accs = []
        for mask in [data.train_mask, data.val_mask, data.test_mask]:
            correct = pred[mask].eq(data.y[mask]).sum().item()
            acc = correct / mask.sum().item()
            accs.append(acc)
        return accs  # [train_acc, val_acc, test_acc]

In [ ]:
embedding_layer = torch.nn.Linear(data.num_node_features, in_channels).to(device)



In [ ]:
for epoch in range(1, 2001):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if epoch % 100 == 0:
        print(f"Epoch {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}")



In [ ]:
print("Unique classes in labels:", data.y.unique())
print("Max label value:", data.y.max().item())
